In [5]:
import pandas as pd
import numpy as np
from pathlib import Path

In [6]:
from vae_pipeline import run_vae_pipeline
import preprocess

In [7]:
RAW_DIR = Path("../data/raw")
PROCESSED_DIR = Path("../data/processed")

### データ作成

In [8]:
preprocess.prepare_npz_data(
    logdata_filepath = RAW_DIR/"T1105/security2_structured.csv",
    output_dir = PROCESSED_DIR/"T1105",
    window_size = 100,
    step_size = 50,
    mode = "fixed", 
    features = ["EventId"]
)

there are 3555 instances (sliding windows) in this dataset

Sequence lengths are consistent (len=100). No padding needed.
[0.8] training size 2828 -> ../data/processed/T1105/ratio_0.8/train.npz
[0.8] test normal size 708 -> ../data/processed/T1105/ratio_0.8/test_normal.npz
[0.8] test abnormal size 19 -> ../data/processed/T1105/ratio_0.8/test_abnormal.npz


### train

In [ ]:
run_vae_pipeline(
    dataset_file = "Integrated/ratio_0.8/train", # 拡張子不要
    vae_type = "timevae",
)

TypeError: run_vae_pipeline() got an unexpected keyword argument 'model_name'